# Goal
The goal of this notebook is to optimize SmartCache. 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# TODO
Lots to do here.

For once - this is probably not the workflow we want. We likely want to:
1. <s>Start the Optimizer in a separate process so that we can monitor its progress. Maybe we should have a separate notebook just for Optimizer Monitoring.</s>
1. Start the Mlos.Agent + Workload in a separate process. Perhaps communicate with them over gRPC. This would bring us closer to the C# scenario but is not a top priority.
1. We need to be able to see how well the cache is doing: what are its stats.
1. We want to modify the workload so that one configuration dominates.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import os
from subprocess import Popen, CREATE_NEW_CONSOLE
import sys
from threading import Thread

import grpc
import pandas as pd

In [ ]:
from mlos.Grpc.BayesianOptimizerFactory import BayesianOptimizerFactory
from mlos.Grpc.BayesianOptimizerProxy import BayesianOptimizerProxy
from mlos.Grpc.OptimizerMonitor import OptimizerMonitor

from mlos.Logger import create_logger

from mlos.Examples.SmartCache import SmartCacheWorkloadGenerator, SmartCache
from mlos.Examples.SmartCache.TelemetryAggregators.WorkingSetSizeEstimator import WorkingSetSizeEstimator

from mlos.Grpc.OptimizerMicroserviceServer import OptimizerMicroserviceServer

from mlos.Mlos.Infrastructure import CommunicationChannel, SharedConfig
from mlos.Mlos.SDK import mlos_globals, MlosGlobalContext, MlosExperiment, MlosAgent
from mlos.Mlos.SDK.CommonAggregators.Timer import Timer

from mlos.Optimizers.BayesianOptimizer import BayesianOptimizerConfig
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Spaces import Point, SimpleHypergrid, ContinuousDimension

import mlos.global_values as global_values

grpc_port = 50051
mlos_python_path = os.path.abspath(os.path.join(os.getcwd(), "..", "Mlos.Python"))

In [ ]:
os.getpid()

In [ ]:
# Let's stand up the Optimizer Service
#
python_path = sys.executable
optimizer_microservice_launcher_path = os.path.abspath(os.path.join(mlos_python_path, "mlos", "start_optimizer_microservice.py"))

command = f"{python_path} {optimizer_microservice_launcher_path} launch --port {grpc_port}"
print(command)
server_process = Popen(command, creationflags=CREATE_NEW_CONSOLE)

In [ ]:
logger = create_logger('Optimizing Smart Cache')
optimizer_service_grpc_channel = grpc.insecure_channel(f'localhost:{grpc_port}')
bayesian_optimizer_factory = BayesianOptimizerFactory(grpc_channel=optimizer_service_grpc_channel, logger=logger)

In [ ]:
# Optimization Problem
#
optimization_problem = OptimizationProblem(
    parameter_space=SmartCache.parameter_search_space,
    objective_space=SimpleHypergrid(name="objectives", dimensions=[ContinuousDimension(name="miss_rate", min=0, max=1)]),
    context_space=None, # TODO add the working set size estimators.
    objectives=[Objective(name="miss_rate", minimize=True)]
)

In [ ]:
optimizer = bayesian_optimizer_factory.create_remote_optimizer(
    optimization_problem=optimization_problem,
    optimizer_config=BayesianOptimizerConfig.DEFAULT
)

In [ ]:
# Now onto the mlos global context - this is where the smart component can send its telemetry and check for new config.
# Note that the communication channel and shared config are - well - shared between mlos agent and the smart component.
#
global_values.declare_singletons()
communication_channel = CommunicationChannel()
shared_config = SharedConfig()
mlos_globals.init_mlos_global_context()
mlos_globals.mlos_global_context = MlosGlobalContext(communication_channel=communication_channel, shared_config=shared_config)
# Now let's create the MlosAgent. Note that this architecture mirrors our native architecture, except it's much less performant.
#

mlos_agent = MlosAgent(
    logger=logger,
    communication_channel=communication_channel,
    shared_config=shared_config,
    mlos_service_endpoint=None, # TODO: remove this completely from everywhere.
    bayesian_optimizer_grpc_channel=optimizer_service_grpc_channel
)

mlos_agent_thread = Thread(target=mlos_agent.run)
mlos_agent_thread.start()

In [ ]:
# Let's register SmartCache as a legal smart component.
#
mlos_agent.add_allowed_component_type(SmartCache)
mlos_agent.add_allowed_component_type(SmartCacheWorkloadGenerator) # TODO: for the very first exercise we should have a fixed workload.

In [ ]:
smart_cache_workload = SmartCacheWorkloadGenerator(logger=logger)

In [ ]:
# Here we define the telemetry aggregators used in the experiments.
#
working_set_size_estimator = WorkingSetSizeEstimator()


# TODO: this should be yet another telemetry aggregator, not a standalone function.
#
def set_new_cache_configuration(elapsed_time_ms):
    # TODO: remove the globals.
    global logger
    global working_set_size_estimator
    global optimizer
    global mlos_agent
    
    # Just to make sure the optimizer monitor and the tomograph are working: let's register some dummy results.
    # TODO: aggregate telemetry to determine the true miss rate for this cache for a given config.
    #
    existing_config = mlos_agent.get_configuration(component_type=SmartCache)
    if existing_config is not None:
        features_df = existing_config.to_pandas()

        if existing_config.implementation == "MRU":
            miss_rate = -1000 + existing_config.mru_cache_config.cache_size
        else:
            miss_rate = 1000 + existing_config.lru_cache_config.cache_size

        objectives_df = pd.DataFrame({'miss_rate': [miss_rate]})
        optimizer.register(features_df, objectives_df)    
    
    # TODO: have the working set size estimate be part of the context passed to the optimizer.
    #
    current_estimate = working_set_size_estimator.estimate_working_set_size()
    logger.info(f"Estimated working set size: {current_estimate.chapman_estimator}")
    
    new_config_values = optimizer.suggest()
    mlos_agent.set_configuration(
    component_type=SmartCache,
        new_config_values=new_config_values
    )
    

# TODO: we need aggregators to spit out hit rate, as well as other Cache Info (cache entry staleness, cache saturation, etc.)

cache_config_timer = Timer(
    timeout_ms=200,
    observer_callback=set_new_cache_configuration
)

In [ ]:
# Now finally we build the experiment.
#
smart_cache_experiment = MlosExperiment(
    smart_component_types=[SmartCache],
    telemetry_aggregators=[cache_config_timer, working_set_size_estimator]
)

In [ ]:
# We have the experiment, let's start it.
#
smart_cache_workload_thread = Thread(target=smart_cache_workload.run, args=(150,))
smart_cache_workload_thread.start()
mlos_agent.start_experiment(smart_cache_experiment)

smart_cache_workload_thread.join()
mlos_agent.stop_experiment(smart_cache_experiment)
mlos_globals.mlos_global_context.stop_clock()

In [ ]:
# Clean up
#
mlos_globals.mlos_global_context.stop_clock()
mlos_agent.stop_all()


In [ ]:
# Kill the server.
#
server_process.kill()